In [4]:
# the purpose here is to get the region of interest of the extracted e's
# and calculate the GLCM of the pixels slices received, 

# acc. to paper the pixels recieved are 180x160, but we receive 14x14, lol so that's one thing
# output preferebly will be a feature vector having the GLCM features for this one pic

In [2]:
# A. Marginal Probabalities - GLCM - DONE
#     a. mean of row
#     b. mean of column
#     c. variance of row
#     d. variance of column
# B. Direct GLCM Metrics - GLCM - DONE
#     a. Energy (sum of squares of marginal probs) E 
#     b. hxy1: Cross entropy using marginal product CEn
#     c. hxy2: Entropy of marginal product MEn
#     d. hglcm: Entropy of the GLCM itself EN
#     e. Max prob in GLCM values P_max
#     f. correlation C
#     g. diagonal correlation C_d
# C. From Difference Matrices - Diff GLCM - DONE
#     a. Energy E_d
#     b. Entropy En_d
#     c. Inertia I_d
#     d. Local Homogenteity H_d
# D. From Sum Matrices - Sum GLCM - DONE
#     a. Energy E_s
#     b. Entropy En_s
#     d. Variance V_s
#     d. cluster shade
#     e. cluster prominence

In [3]:
import cv2
import pytesseract
import numpy as np
from PIL import Image, ImageDraw
import os

# GLCM
from skimage.feature import graycomatrix, graycoprops

In [4]:
# def get_e(image_path, n, crop_size, op_dir):
    
#     # Load the image
#     # image_path = 'test_images/test_0.png'  # Replace with your image
#     img1 = cv2.imread(image_path)
#     if img1 is None:
#         raise ValueError(f"Image at path {image_path} could not be loaded.")
            
#     img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
#     #upscalling images, solely to help tesseract, image extraction will be done from the og image
#     # img = cv2.resize(img2, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
#     img = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_CUBIC)   # this is the processed image which 
#                                                                                            # we put in the tesseract
#                                                                                            # coz it needs clearer pcitures to 
#                                                                                            # accurately locate the e's
#     # print(img.shape, img2.shape)
#     half = crop_size//2
    
#     # Get image height (Tesseract origin is bottom-left)
#     h, w = img.shape
    
#     # op_dir = 'saved_e'
#     os.makedirs(op_dir, exist_ok=True)
    
#     # Draw setup
#     # draw = ImageDraw.Draw(img)
    
#     # Get character bounding boxes
#     boxes = pytesseract.image_to_boxes(img)
#     count = 0

#     # Draw green dots on top of each 'e'
#     for b in boxes.strip().splitlines():
#         b = b.split()
#         char, x1, y1, x2, y2 = b[0], int(b[1]), int(b[2]), int(b[3]), int(b[4])
        
#         # Flip y-coordinates
#         y1_new = h - y1
#         y2_new = h - y2
    
#         # make a bounding box around the e
#         # cv2.rectangle(img, (x1, y2_new), (x2, y1_new), (255, 0, 0), 1)
    
#         if char.lower() == 'e' and count<n:
    
#             # compute center of bounding box    
#             cx = (x1 + x2) // 2
#             cy = (y1_new + y2_new) // 2
        
#             # Get top-left and bottom-right coordinates of fixed crop
#             x_start = max(0, cx - half)
#             y_start = max(0, cy - half)
#             x_end = min(img2.shape[1], cx + half)
#             y_end = min(img2.shape[0], cy + half)
    
#             # crop image
#             cropped = img2[y_start:y_end, x_start:x_end]
    
#             # if dimensions okay, at image to output dir, 
#             # later make sure that the image dirs are constant, else will cause computation error
#             if cropped.shape[0] > 0 and cropped.shape[1] > 0:
#                 out_path = os.path.join(op_dir, f"e_{count+1}.png")
#                 cv2.imwrite(out_path, cropped)
#                 count += 1

#     cv2.imwrite("output_with_boxes.png", img)
#     print(f"Saved {count} 'e' characters in '{op_dir}/' and annotated image as 'output_with_boxes.png'")
    
#     # # to check the dimensions of the images
#     # for filename in os.listdir(op_dir):
#     #     full_path = os.path.join(op_dir, filename)
#     #     img_saved = cv2.imread(full_path)
        
#     #     if img_saved is not None:
#     #         print(f"{filename}: {img_saved.shape}")
#     #     else:
#     #         print(f"Failed to read {filename}")

#     return 0

In [5]:
# get region of interest

# threshold -> if sufficiently black, it is counted as ROI
def roi(img, threshold):
    if len(img.shape) != 2:
        raise ValueError("Input image must be grayscale")
    R = np.sum(img < threshold)
    
    # print(R)
    return R

In [6]:
# lets get the dimensions correct here
# GLCM's are inherenetly 4Ds - distance, angles, nxm pairs and number of pair occurances
# we are eliminating angle, hence we get a 3D vector -> (G,G,F), G=levels, D=distances
# 3D vector (GLCM GxGxD) -> normalise -> 2 x row/column sums (1 x 10)-> 2 x mean and variance of each row/column sum(1 x scalar)
# total = 4 features

In [7]:
def glcm(img_path, dist, levels):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Quantize to 8 gray levels (0 to 7)
    levels = 256
    # quantized = (gray / (256 // levels)).astype(np.uint8)

    distance = list(range(1, dist+1))
    # R = roi(gray, 256)
    
    # Compute GLCM
    glcm = graycomatrix(gray, distances=distance, angles=[0], levels=levels, symmetric=False, normed=True)
    
    # display GLCM matrix for angle=0 and distance=1
    # print(glcm[:, :, 0, 0])
    n_glcm = glcm[:,:,:,0]  #//R 
    # print(n_glcm)
    return n_glcm

In [8]:
def get_sum(arr,ch):
    # print(arr.shape)
    if ch == 'c':
        c = np.sum(arr, axis=0, keepdims=True)
        # print("Shape is:", c.shape)
        return c 
    elif ch == 'r':
        r = np.sum(arr, axis=1, keepdims=True) 
        # print("Shape is:", r.shape)
        return r

In [9]:
def get_var(arr, mean):
    m, n, d = arr.shape

    if m!=1 and n ==1: 
        a = m
        axis = 0
    elif n!=1 and m ==1: 
        a = n
        axis = 1
    else: print("not valid state")

    var = np.zeros((1,1,d))
    mean = mean.flatten()
    
    for i in range(d):
        summ = 0
        for j in range(a):
            val = arr[j, 0, i] if axis == 0 else arr[0, j, i]
            summ += (j ** 2) * val
        var[0, 0, i] = summ - mean[i]
    return var

In [12]:
if __name__ == "__main__":
    img_path = "saved_e/2/e_16.png"  

    N = 256
    # get probablities from GLCM, this is nromalised with ROI
    p_row = get_sum(glcm(img_path, 10, N), 'r')
    p_col = get_sum(glcm(img_path, 10, N), 'c')
    print("Row Sums:", p_row) 
    print("Columns Sums:", p_col) 

    # added all the probablities such that the matrix is 1 scalar x no. of disances, (1 x 1 x 10)
    u_row = get_sum(p_row,'c')
    u_col = get_sum(p_col,'r')
    # print("r", u_row.shape)
    # print("c", u_col.shape)
    var_r = get_var(p_row, u_row)
    var_c = get_var(p_col, u_col)
    
    print("Mean of Row Sums:", u_row) 

    print("Mean of Column Sums:", u_col )
    print("Variance of Row Sums:",var_r ) # <- v_r
    print("Variance of Column Sums:", var_c) # <- v_c


Row Sums: [[[0.04945055 0.05357143 0.05194805 ... 0.07142857 0.07142857 0.05357143]]

 [[0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]]

 ...

 [[0.01648352 0.01785714 0.01948052 ... 0.03571429 0.04285714 0.03571429]]

 [[0.02197802 0.02380952 0.01948052 ... 0.03571429 0.04285714 0.05357143]]

 [[0.28021978 0.26785714 0.25974026 ... 0.22619048 0.22857143 0.25      ]]]
Columns Sums: [[[0.03846154 0.04166667 0.04545455 ... 0.03571429 0.02857143 0.03571429]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.01648352 0.01785714 0.00649351 ... 0.         0.         0.        ]
  [0.02197802 0.01785714 0.01298701 ... 0.02380952 0.02857143 0.03571429]
  [0.31318681 0.31547619 0.2987013  ... 0.30952381 0.3        0.35714286]]]
Mean of Row Sums: [[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]]
Mean of 

In [ ]:
# there are 7 features computed below, take a look

In [14]:
def get_energy(arr):
    '''input GLCM matirx, normalized'''
    m,n,d = arr.shape
    e = np.zeros(d)
    
    for i in range(d):
        for j in range(m):
            for k in range(n):
                e[i] = e[i] + (arr[j,k,i]**2)
    return e

In [15]:
def get_entropy1(arr, ar, ac):
    '''input GLCM matirx, normalized, mean_row and mean_column'''
    m, n, d = arr.shape
    
    m1, n1, d = ar.shape
    if n1!=1: print("Nope, shape of ar is not correct")
                    
    m2, n2, d = ac.shape
    if m2!=1: print("Nope, chape of ac is not correct")

    # for k in range(d):
    # summ = np.zeros(d)        
    #     for i in range(m1):
    #         for j in range(n2):
    #             summ[i][1][d] = summ[][][d] + arr[i][j][d]*np.log(ar[m][1]*ac[1][n])

    summ = np.zeros(d)
    for k in range(d):
        for i in range(m):
            for j in range(n):
                p = arr[i, j, k]
                pr = ar[i, 0, k]
                pc = ac[0, j, k]
                if p > 0 and pr > 0 and pc > 0:
                    summ[k] += p * np.log(pr * pc)
    
    return -1*summ

In [16]:
def get_entropy2(ar, ac):
    '''input mean row and mean column'''
    m1, n1, d = ar.shape
    if n1!=1: print("Nope, shape of ar is not correct")
                    
    m2, n2, d = ac.shape
    if m2!=1: print("Nope, chape of ac is not correct")

    # for k in range(d):
    # summ = np.zeros(d)        
    #     for i in range(m1):
    #         for j in range(n2):
    #             summ[i][1][d] = summ[][][d] + arr[i][j][d]*np.log(ar[m][1]*ac[1][n])


    summ = np.zeros(d)
    for k in range(d):
        # summ = 0
        for i in range(m1):
            for j in range(n2):
                if ar[i, 0, k] * ac[0, j, k]> 0:
                    summ[k] = summ[k] + ar[i, 0, k] * ac[0, j, k] * np.log(ar[i, 0, k] * ac[0, j, k])
    
                
    return -1*summ

In [17]:
def get_entropy3(arr):
    '''input GLCM matirx, normalized'''
    m, n, d = arr.shape
    
    # for k in range(d):
    # summ = np.zeros(d)        
    #     for i in range(m1):
    #         for j in range(n2):
    #             summ[i][1][d] = summ[][][d] + arr[i][j][d]*np.log(ar[m][1]*ac[1][n])

    summ = np.zeros(d)
    for k in range(d):
        # summ = 0
        for i in range(m):
            for j in range(n):
                if arr[i,j,k]>0:
                    summ[k] = summ[k] + arr[i,j,k]*np.log(arr[i,j,k])
    
    return -1*summ

In [18]:
def max_prob(arr):
    '''input GLCM matirx, normalized'''
    m, n, d = arr.shape

    mP = np.zeros(d)
    for i in range(d):
        mP[i] = np.max(arr[:,:,i])

    return mP

In [19]:
def get_haralicks(arr, ur, uc, var_r, var_c, eps = 1e-8):
    '''input GLCM matirx, normalized, 
    mean row and mean column,
    variance row and variance column,
    and epsilon value to prevent denom form goint 0'''
    m, n, d = arr.shape

    arr = arr.astype(np.float64)
    ur = ur.astype(np.float64)
    uc = uc.astype(np.float64)
    var_r = var_r.astype(np.float64)
    var_c = var_c.astype(np.float64)

    h = np.zeros(d)

    for k in range(d):
        for i in range(m):
            for j in range(n):
                h[k] += (i - ur[0,0,k])*(j - uc[0,0,k])*arr[i,j,k]/(np.sqrt(var_r[0,0,k]*var_c[0,0,k]) + eps)
    
    # print(h)
    return h

In [20]:
def get_dCorr(arr, ur, uc):
    '''input GLCM matirx, normalized,
    mean row and mean column'''
    m,n,d = arr.shape

    arr = arr.astype(np.float64)
    ur = ur.astype(np.float64)
    uc = uc.astype(np.float64)
    
    dC = np.zeros(d)

    for k in range(d):
        mu_r = ur[0, 0, k]
        mu_c = uc[0, 0, k]
        for i in range(m):
            for j in range(n):
                dC[k] += abs(i - j) * (i + j - mu_r - mu_c) * arr[i, j, k]

    return dC

In [21]:
if __name__ == "__main__":
    img_path = "saved_e/2/e_16.png"  

    arr = cv2.imread(img_path)
    N = 256

    E = get_energy(glcm(img_path, 10, N))
    hxy1 = get_entropy1(glcm(img_path, 10, N), p_row, p_col)
    hxy2 = get_entropy2(p_row, p_col)
    hxy3 = get_entropy3(glcm(img_path, 10, N))

    print("Energy:", E)
    print("hxy1: ", hxy1)
    print("hxy2:", hxy2) 
    print("hxy3:", hxy3) 

    # print(glcm(img_path, 10, N).shape)
    # print("ur shape:", u_row.shape)
    # print("uc shape:", u_col.shape)
    # print("var_r shape:", var_r.shape)
    # print("var_c shape:", var_c.shape)

    mP = max_prob(glcm(img_path, 10, N))
    h = get_haralicks(glcm(img_path, 10, N), u_row, u_col, var_r, var_c)
    dC = get_dCorr(glcm(img_path, 10, N), u_row, u_col)

    print("max Prob:", mP)
    print("Correlation coeff:", h)
    print("Diagonal Correlation:", dC)

Energy: [0.01726845 0.01658163 0.01239669 0.01744898 0.01499118 0.01833546
 0.01915868 0.01417234 0.02163265 0.02423469]
hxy1:  [6.97568643 6.93894339 7.00402094 6.92120336 6.69185465 6.54734925
 6.494987   6.45364105 6.27419128 5.81207383]
hxy2: [6.97568643 6.93894339 7.00402094 6.92120336 6.69185465 6.54734925
 6.494987   6.45364105 6.27419128 5.81207383]
hxy3: [4.75050886 4.73824875 4.72375896 4.58488028 4.54621051 4.43348079
 4.30177371 4.30906312 4.01471016 3.82730964]
max Prob: [0.1043956  0.10119048 0.06493506 0.1        0.07936508 0.09821429
 0.09183673 0.03571429 0.07142857 0.07142857]
Correlation coeff: [0.88772359 0.77057382 0.74992904 0.77997953 0.77956869 0.7710333
 0.75612477 0.72569898 0.73310016 0.83269486]
Diagonal Correlation: [16186.93406593 26360.69642857 27399.65584416 24726.52142857
 25513.61904762 26938.99107143 27998.16326531 31466.44047619
 31010.         23192.58928571]


In [22]:
# now we will compute the next 5 features - energy, entropy, inertia and local homogei

In [23]:
def get_D_k(arr):
    ''' output is (max_diff+1, d) array, where each D[k] is a sum over |n - m| = k'''
    
    m, n, d = arr.shape

    md = max(m, n) - 1

    D_k = np.zeros((md+1, d))
    for k in range(d):
        for i in range(m):
            for j in range(n):
                diff = np.abs(i - j)
                D_k[diff, k] += arr[i,j,k]

    return D_k

In [24]:
def get_Denergy(arr):
    return np.sum(arr, axis=0) 

In [25]:
def get_entropy3(arr):
    '''input GLCM matirx, normalized'''
    m, d= arr.shape
    
    # for k in range(d):
    # summ = np.zeros(d)        
    #     for i in range(m1):
    #         for j in range(n2):
    #             summ[i][1][d] = summ[][][d] + arr[i][j][d]*np.log(ar[m][1]*ac[1][n])

    summ = np.zeros(d)
    for k in range(d):
        # summ = 0
        for i in range(m):
                if arr[i,k]>0:
                    summ[k] = summ[k] + arr[i,k]*np.log(arr[i,k])
    
    return -1*summ

In [26]:
def get_inertia(arr):
    
    k = np.arange(D_k.shape[0])[:, np.newaxis]  # shape (256, 1)
    ID = np.sum((k**2) * D_k, axis=0)  # sum over k for each distance
    return ID

In [27]:
def get_hD(D_k):
    
    k = np.arange(D_k.shape[0])[:, np.newaxis]  # shape (256, 1)
    hD = np.sum(D_k / (1 + k**2), axis=0)
    return hD

In [28]:
if __name__ == "__main__":

    img_path = "saved_e/2/e_16.png"  

    # img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # print(glcm(img_path, 10, 256).shape)

    D_k = get_D_k(glcm(img_path, 10, 256))
    print(D_k.shape)
    D_En = get_Denergy(D_k)
    DEnt = get_entropy3(D_k)
    Id = get_inertia(D_k)
    hD = get_hD(D_k)
    
    print("D(k) matrix:", D_k[0])
    print("D(k)Energy:", D_En)
    print("D Entropy", DEnt)
    print("D Inertia", Id)
    print("D Local Homgenity", hD)


(256, 10)
D(k) matrix: [0.12087912 0.11309524 0.08441558 0.12857143 0.07936508 0.10714286
 0.12244898 0.02380952 0.08571429 0.07142857]
D(k)Energy: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
D Entropy [4.04064073 4.21219259 4.12430762 4.09596694 4.1017492  4.03281548
 4.01538917 3.95407646 3.81765699 3.57648118]
D Inertia [ 8672.47252747 18018.80357143 19143.14935065 16953.69285714
 17548.28571429 18413.58035714 19682.30612245 22203.51190476
 21812.51428571 14404.69642857]
D Local Homgenity [0.14476379 0.12928715 0.11428776 0.15099597 0.10807036 0.12748367
 0.14200942 0.04413208 0.1124399  0.1183941 ]


In [29]:
# now we are going to calculate the last 5 features

In [30]:
def get_S_k(arr):
    ''' output is (max_diff+1, d) array, where each D[k] is a sum over |n - m| = k'''
    
    m, n, d = arr.shape

    ms = m+n-2

    S_k = np.zeros((ms+1, d))
    for k in range(d):
        for i in range(m):
            for j in range(n):
                summ = i + j
                S_k[summ, k] += arr[i,j,k]

    return S_k

In [31]:
def mean_S(S_k):
    ks = np.arange(S_k.shape[0])  # array of k from 0 to 510
    mu_S = np.sum(ks[:, None] * S_k, axis=0)  # shape: (d,)
    return mu_S.reshape(1, -1)  # shape (1, 10)


In [32]:
def get_sigma(S_k, mu_S):
    k = np.arange(511).reshape(-1, 1)  # shape (512, 1)
    result = np.sum(((k - mu_S) ** 2) * S_k, axis=0)
    return result.reshape(1, -1)  # shape (1, 10)


In [33]:
def get_A_D(S_k, mu_r, mu_c, sigma_r, sigma_c):
    k = np.arange(511).reshape(-1, 1)
    numerator = ((k - mu_r - mu_c) ** 3) * S_k
    denominator = (sigma_r**2 + sigma_c**2 + 2 * sigma_r * sigma_c) ** (3 / 2)
    result = np.sum(numerator, axis=0) / denominator
    return result.reshape(1, -1)

In [34]:
def get_B_D(S_k, mu_r, mu_c, sigma_r, sigma_c):
    k = np.arange(511).reshape(-1, 1)
    numerator = ((k - mu_r - mu_c) ** 4) * S_k
    denominator = (sigma_r**2 + sigma_c**2 + 2 * sigma_r * sigma_c) ** 2
    result = np.sum(numerator, axis=0) / denominator
    return result.reshape(1, -1)

In [37]:
if __name__ == "__main__":

    img_path = "saved_e/2/e_12.png"  

    # img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # print(glcm(img_path, 10, 256).shape)

    S_k = get_S_k(glcm(img_path, 10, 256))
    print(S_k.shape)
    S_En = get_Denergy(S_k)
    print(S_En.shape)
    SEnt = get_entropy3(S_k)
    print(SEnt.shape)
    u_s = mean_S(S_k)
    print(u_s.shape)
    s = get_sigma(S_k, u_s)
    A_d = get_A_D(S_k, u_row, u_col, np.sqrt(var_r), np.sqrt(var_c))
    B_d = get_B_D(S_k, u_row, u_col, np.sqrt(var_r), np.sqrt(var_c))
    
    print("S(k) matrix:", D_k[0])
    print("S(k)Energy:", D_En)
    print("S Entropy", DEnt)
    print("S mean", u_s)
    print("S Variance", s)
    print("S cluster shade", A_d)
    print("S cluster prominence", B_d)

(511, 10)
(10,)
(10,)
(1, 10)
S(k) matrix: [0.12087912 0.11309524 0.08441558 0.12857143 0.07936508 0.10714286
 0.12244898 0.02380952 0.08571429 0.07142857]
S(k)Energy: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
S Entropy [4.04064073 4.21219259 4.12430762 4.09596694 4.1017492  4.03281548
 4.01538917 3.95407646 3.81765699 3.57648118]
S mean [[373.82967033 364.70238095 358.81168831 363.78571429 373.81746032
  378.33928571 381.90816327 386.66666667 396.47142857 427.21428571]]
S Variance [[30351.4709878  22766.85189909 18953.26973351 18309.99693878
  19356.03810784 20178.86702806 19576.08340275 15969.1031746
  12533.33489796 10011.66836735]]
S cluster shade [[-6.46315670e-10  0.00000000e+00 -1.64703580e-09 ...  1.85351924e-01
   2.18877046e-01  2.75362407e-01]]
S cluster prominence [[3.16677137e-12 0.00000000e+00 8.27499284e-12 ... 2.31415564e-01
  2.71812636e-01 3.29499458e-01]]
